In [71]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
import math

#Importing Iris dataset from sklearn
iris=datasets.load_iris()

#data
X_train = iris.data

#appending features as 0 based index 
features = []
for each in range(X_train.shape[1]):
    features.append(each)
    

#actual preditions
y_ = iris.target
y_train = y_

#Spliting data and actual prediction i.e df and y into training and testing data with 70-30 ratio
# X_train,X_test,y_train,y_test = train_test_split(df,y_,test_size=0.3,random_state=1)
print(set(y_train))
# y_train
iris.target_names[0]

{0, 1, 2}


'setosa'

In [16]:
#printing shapes of X_train, y_train, X_test, y_test to get idea of size of data we had...
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

In [38]:
#function to count frequency of label/classes and store in a dictionary
def count_freq(y):
    classes_freq = {}
#     print(y)
    for i in y:
#         classes_freq[i] += 1
        if i not in classes_freq:
            classes_freq[i] = 1
        else:
            classes_freq[i] += 1
#     print(classes_freq)
    return classes_freq

In [39]:
#function to find entropy of split
def entropy_(y):
    hash_ = count_freq(y)
    total = len(y)
    entropy = 0
    for each in hash_:
        prob = hash_[each]/total
        entropy += (-prob)*math.log2(prob)
    return entropy

In [48]:
#funtion to calculate gain ratio of split
def gain_ratio(X,y,feature_id):
    #finding entropy before splitiing
    info_orginal = entropy_(y)
    #extracting unquie labels from feature id 
    unq = set(X[:,feature_id])
    #collecting features and y together
    data=pd.DataFrame(X)
    data[data.shape[1]] = y
    #calculating total number of sample data/data points
    total = len(y)
    #varibale to store information gain after splitting
    info_f=0
    #variable to store split info
    split_info = 0
    #iterating through each unqiue label in feature id 
    for each in unq:
        #collecting only unqiue label X and Y
        df = data[data[feature_id]==each]
        tot = df.shape[0]
        y_values = df[df.shape[1]-1]
        info_f += (tot/total)*(entropy_(y_values))
        split_info += ((-tot/total)*math.log2(tot/total))
    #checking if split info is zero or not, if zero then it will be undefined after dividing it by information gain
    if split_info == 0:
        return 1000000000
    info_gain = info_orginal - info_f
    #calculating gain ratio
    gain_ratio_ = info_gain/split_info
    return gain_ratio_
    

In [49]:
print(features)

[0, 1, 2, 3]


In [74]:
#Decision tree steps using recursion
def Decision_tree_steps(X,y,features,cnt,classes):
    #if tree node contains only one class
    if len(set(y))==1:
        print("Level ",cnt)
        for i in classes:
            if i in y:
                print("Count of ",i,"= ",len(y))
            else:
                print("Count of ",i,"= ",0)
        print("Current Entropy is = 0.0")
        print("Leaf node")
        print()
        return
    #if we are out of features to split
    if len(features) == 0:
        print("Level ",cnt)
        hash_ = count_freq(y)
        max_ = -math.inf
        for i in classes:
            if i not in hash_:
                print("Count of ",i,"= ",0)
            else:
                max_ = max(hash_[i],max_)
                print("Count of ",i,"= ",hash_[i])
        print("Current Entropy is = ",entropy_(y))
        print("Lead node")
        print()
        return
    
    #variable to store maximum gain ratio
    max_gain = -math.inf
    feature_to_choose = None
    for feature in features:
        current_gain = gain_ratio(X,y,feature)
        #if current gain ratio  greater than max gain ratio so far, then update max gain ratio 
        if current_gain > max_gain:
            max_gain = current_gain
            feature_to_choose = feature
    
    print("Level ",cnt)
    hash_ = count_freq(y)
    max_ = -math.inf
    
    #printing entropy and gain ratio 
    for class_ in classes:
        if class_ not in hash_:
            print("Count of ",class_,"= ",0)
        else:
            max_ = max(hash_[class_],max_)
            print("Count of ",class_,"= ",hash_[class_])
    print("Current Entropy is =",entropy_(y))
    print("Splitting on feature  X[",feature_to_choose,"] with gain ratio ",max_gain,sep="")
    print()
    
    #extracting unqiue labels from choosen feature
    extract_unique = set(X[:,feature_to_choose])
    #combining X and y for furthur spliting X and y according to unqiue feature label
    data = pd.DataFrame(X)
    data[data.shape[1]] = y
    
    #removing choosen feature since, we don't want to split data with this again
    index = features.index(feature_to_choose)
    features.remove(feature_to_choose)
    
    #calling decision tree step furthur for remaining features...
    for each in extract_unique:
        df = data[data[feature_to_choose]==each]
        x_ = df.iloc[:,0:df.shape[1]-1]
        y_ = df.iloc[:,df.shape[1]-1]
        Decision_tree_steps(x_,y_,features,cnt+1,classes)
        

In [77]:
#pre computing unique class from y
classes = set(y_train)
print(classes)
Decision_tree_steps(X_train,y_train,features,1,classes)

{0, 1, 2}
Level  1
Count of  0 =  50
Count of  1 =  50
Count of  2 =  50
Current Entropy is = 1.584962500721156
Splitting on feature  X[1] with gain ratio 0.12841651229496967

Level  2
Count of  0 =  1
Count of  1 =  7
Count of  2 =  2
Current Entropy is =  1.1567796494470395
Lead node

Level  2
Count of  0 =  6
Count of  1 =  8
Count of  2 =  12
Current Entropy is =  1.5262349099495225
Lead node

Level  2
Count of  0 =  6
Count of  1 =  0
Count of  2 =  0
Current Entropy is = 0.0
Leaf node

Level  2
Count of  0 =  5
Count of  1 =  3
Count of  2 =  5
Current Entropy is =  1.5485806065228545
Lead node

Level  2
Count of  0 =  3
Count of  1 =  0
Count of  2 =  1
Current Entropy is =  0.8112781244591328
Lead node

Level  2
Count of  0 =  4
Count of  1 =  3
Count of  2 =  4
Current Entropy is =  1.5726236638951638
Lead node

Level  2
Count of  0 =  0
Count of  1 =  0
Count of  2 =  0
Current Entropy is = 0.0
Leaf node

Level  2
Count of  0 =  9
Count of  1 =  1
Count of  2 =  2
Current Ent